<h1> Eyes of Hajj </h1>
<h3>Hajj Crowd Abnormal Behavior Detection</h3>
<h3>Project Workflow</h3>
<ol>
    <li>Loading the HAJJv2 Dataset.</li>
    <li>Extracting the images (frames) from the videos and merge the labesls into a single file for train & test.</li>
    <li>Transforming the data to Yolo format where we optain a text file (label) for each image in train & test & val. </li>
    <li>Changeing the Yolo feature extractor's backbone to MobileNet/ResNet50 to optimize the feature extraction.</li>
    <li>Applaying the Lucas-Kanade algorithm on the frames (images) from the original dataset to estimate the optical flow (Magnitude & Orientation/Direction).</li>
    <li>Taking the mean & varience & STD of the optical flow and feed it into the Random Forest classifier to predect the actual class of the abnormal behavior.</li>
    <li>Visualizing the Random Forest's predections on the testing data.</li>
    <li>Evaluating the models' performences on the testing data.</li>
</ol>

In [1]:
# Importing the required libraries

import pandas as pd
import os
import shutil

In [2]:
# Load labels data
Train_Labels = pd.read_csv('HAJJv2_Dataset/Extracted_Data/Train_Labels.csv')
Test_Labels = pd.read_csv('HAJJv2_Dataset/Extracted_Data/Test_Labels.csv')

In [3]:
# Load iamges data
Train_Image_path = 'HAJJv2_Dataset/Extracted_Data/Train/'
Test_Image_path = 'HAJJv2_Dataset/Extracted_Data/Test/'

### 3. Transforming the data to Yolo format where we optain a text file (label) for each image in train & test & val:

In [4]:
# Create training data folder
os.makedirs('HAJJv2_Dataset/Yolo_Data/Train' , exist_ok= True)
os.makedirs('HAJJv2_Dataset/Yolo_Data/Train/Images' , exist_ok= True)
os.makedirs('HAJJv2_Dataset/Yolo_Data/Train/Labels' , exist_ok= True)

# Create validation data folder
os.makedirs("HAJJv2_Dataset/Yolo_Data/Val" , exist_ok= True)
os.makedirs('HAJJv2_Dataset/Yolo_Data/Val/Images' , exist_ok= True)
os.makedirs('HAJJv2_Dataset/Yolo_Data/Val/Labels' , exist_ok= True)

# Create testing data folder
os.makedirs("HAJJv2_Dataset/Yolo_Data/Test" , exist_ok= True)
os.makedirs('HAJJv2_Dataset/Yolo_Data/Test/Images' , exist_ok= True)
os.makedirs('HAJJv2_Dataset/Yolo_Data/Test/Labels' , exist_ok= True)

In [5]:
Train_Labels.Classes.unique()

array(['sitting', 'standing', 'moving_in_opposite',
       'moving_non_human_object', 'sleeping', 'different_crowd_direction',
       'running'], dtype=object)

In [6]:
dict = {"moving_in_opposite" : 0, "standing" : 1, "different_crowd_direction": 2, "sleeping" : 3, "sitting": 4, "moving_non_human_object":5 ,
        "running": 6 }
train_labels=Train_Labels.replace({"Classes": dict})

In [7]:
Train_Labels

,Video_No,Frame_No,X,Y,Width,Height,Classes,Obj_Name,image_path
0,10,1,130,1020,101,61,sitting,sitting1,10_1.jpg
1,10,1,171,981,102,97,sitting,sitting2,10_1.jpg
2,10,1,219,965,97,111,sitting,sitting3,10_1.jpg
3,10,1,239,871,120,140,sitting,sitting4,10_1.jpg
4,10,1,273,858,93,93,sitting,sitting5,10_1.jpg
...,...,...,...,...,...,...,...,...,...
170767,9,700,1170,114,16,19,sitting,sitting69,9_700.jpg
170768,9,700,1174,108,16,21,sitting,sitting70,9_700.jpg
170769,9,700,1198,71,14,21,sitting,sitting71,9_700.jpg
170770,9,700,1392,262,26,25,sitting,sitting72,9_700.jpg


In [8]:
Test_Labels=Test_Labels.replace({"Classes": dict})

In [9]:
Test_Labels

,Video_No,Frame_No,X,Y,Width,Height,Classes,Obj_Name,image_path
0,10,1,130,1012,101,69,4,sitting1,10_1.jpg
1,10,1,171,981,102,97,4,sitting2,10_1.jpg
2,10,1,219,965,97,111,4,sitting3,10_1.jpg
3,10,1,239,871,120,140,4,sitting4,10_1.jpg
4,10,1,277,856,89,116,4,sitting5,10_1.jpg
...,...,...,...,...,...,...,...,...,...
170526,9,475,1170,114,16,19,4,sitting69,9_475.jpg
170527,9,475,1174,108,16,21,4,sitting70,9_475.jpg
170528,9,475,1198,71,14,21,4,sitting71,9_475.jpg
170529,9,475,1392,262,26,25,4,sitting72,9_475.jpg


In [10]:
Width_image = 1440
height_image = 1080

In [11]:
# 80% of train for train

Train = Train_Labels[:136618]

for i , row in Train.iterrows():

    width = row["Width"]/Width_image
    height = row["Height"]/height_image

    X = row["X"]/Width_image
    Y = row["Y"] /height_image 

    x_center = X+(width/2)
    y_center = Y+(height/2)
    
    cls = row['Classes']
    name = row['image_path'].split('.')[0]
    # save the new annotation results into a txt files
    with open(f'HAJJv2_Dataset/Yolo_Data/Train/Labels/{name}.txt' , 'a') as f : # 'a' append mode to make one txt file per image
        f.write(str(cls) + ' ' + str(x_center) + ' ' + str(y_center) + ' ' +  str(width) + ' ' + str(height) + '\n' )

# copy the Training images to Training folder
Training_no_dub = Train.drop_duplicates(subset = ['image_path'])
for i ,  row in Training_no_dub.iterrows():
    name = row['image_path'].split('.')[0]
    shutil.copy(f'HAJJv2_Dataset/Extracted_Data/Train/{name}.jpg',
                f'HAJJv2_Dataset/Yolo_Data/Train/Images/{name}.jpg')

In [12]:
# 20% of train for val

Val = Train_Labels[136619:]

for i , row in Val.iterrows():

    width = row["Width"]/Width_image
    height = row["Height"]/height_image

    X = row["X"]/Width_image
    Y = row["Y"] /height_image 

    x_center = X+(width/2)
    y_center = Y+(height/2)

    cls = row['Classes']
    name = row['image_path'].split('.')[0]
    # save the new annotation results into a txt files
    with open(f'HAJJv2_Dataset/Yolo_Data/Val/Labels/{name}.txt' , 'a') as f : # 'a' append mode to make one txt file per image
        f.write(str(cls) + ' ' + str(x_center) + ' ' + str(y_center) + ' ' +  str(width) + ' ' + str(height) + '\n' )

# copy the training images to training folder
Training_no_dub = Val.drop_duplicates(subset = ['image_path'])
for i ,  row in Training_no_dub.iterrows():
    name = row['image_path'].split('.')[0]
    shutil.copy(f'HAJJv2_Dataset/Extracted_Data/Train/{name}.jpg',
                f'HAJJv2_Dataset/Yolo_Data/Val/Images/{name}.jpg')

In [13]:
# 100% of test for test

Test = Test_Labels[:]

for i , row in Test.iterrows():

    width = row["Width"]/Width_image
    height = row["Height"]/height_image

    X = row["X"]/Width_image
    Y = row["Y"] /height_image 

    x_center = X+(width/2)
    y_center = Y+(height/2)

    cls = row['Classes']
    name = row['image_path'].split('.')[0]
    # save the new annotation results into a txt files
    with open(f'HAJJv2_Dataset/Yolo_Data/Test/Labels/{name}.txt' , 'a') as f : # 'a' append mode to make one txt file per image
        f.write(str(cls) + ' ' + str(x_center) + ' ' + str(y_center) + ' ' +  str(width) + ' ' + str(height) + '\n' )

# copy the training images to training folder
training_no_dub = Test.drop_duplicates(subset = ['image_path'])
for i ,  row in training_no_dub.iterrows():
    name = row['image_path'].split('.')[0]
    shutil.copy(f'HAJJv2_Dataset/Extracted_Data/Test/{name}.jpg',
                f'HAJJv2_Dataset/Yolo_Data/Test/Images/{name}.jpg')

### 4. Changeing the Yolo feature extractor's backbone to MobileNet/ResNet50 to optimize the feature extraction:

In [ ]:
!python train.py --data data.yaml --cfg yolov5-mobilenet.yaml --batch-size -1 --epochs 300 --cache